In [1]:
import pandas as pd
import xml.etree.ElementTree as xet

In [2]:
#importing the required libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
from glob import glob

In [4]:
df = pd.read_csv('latest_test_data_info.csv', names=['data'])
path = df['data'].tolist()
path.pop(0)
print(path[:5])

['./images\\N209.xml', './images\\N70.xml', './images\\N57.xml', './images\\N174.xml', './images\\N164.xml']


In [5]:
#Extracting the coordinates of number plate from the xml files
test_labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filePath in path:
    #firstImagePath = path[0]
    info = xet.parse(filePath)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)
    #print(xmin , xmax , ymin , ymax)
    test_labels_dict['filepath'].append(filePath)
    test_labels_dict['xmin'].append(xmin)
    test_labels_dict['xmax'].append(xmax)
    test_labels_dict['ymin'].append(ymin)
    test_labels_dict['ymax'].append(ymax)

In [6]:
#converting Dictionary to Dataframe
df = pd.DataFrame(test_labels_dict)
display(df.head())

,filepath,xmin,xmax,ymin,ymax
0,./images\N209.xml,145,319,200,244
1,./images\N70.xml,186,395,376,430
2,./images\N57.xml,211,406,161,202
3,./images\N174.xml,315,665,442,531
4,./images\N164.xml,248,421,121,174


In [17]:
df.shape

(45, 5)

In [7]:
#function that returns image file path when given its corresponding xml file path
def getImgFilePath(xmlFilePath):
    fileName = xet.parse(xmlFilePath).getroot().find('filename').text
    imgFilePath = os.path.join('./images',fileName)
    return imgFilePath

In [8]:
images_path = list(df['filepath'].apply(getImgFilePath))
images_path[:5]

['./images\\N209.jpeg',
 './images\\N70.jpeg',
 './images\\N57.jpeg',
 './images\\N174.jpeg',
 './images\\N164.jpeg']

In [9]:
#verification
sample_img_path = images_path[0]
sample_img_path

'./images\\N209.jpeg'

In [10]:
#displaying the image
sample_img = cv2.imread(sample_img_path)
cv2.namedWindow('SampleImageWindow' , cv2.WINDOW_NORMAL)
cv2.imshow('SampleImage' , sample_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
from tensorflow.keras.preprocessing.image import load_img , img_to_array

In [12]:
labels = df.iloc[:,1:].values
labels[:5]

array([[145, 319, 200, 244],
       [186, 395, 376, 430],
       [211, 406, 161, 202],
       [315, 665, 442, 531],
       [248, 421, 121, 174]], dtype=int64)

In [13]:
data = []
output = []
original_dim = []
for index in range(len(images_path)):
    image_path = images_path[index]
    img_arr = cv2.imread(image_path)
    h,w,d = img_arr.shape
    original_dim.append([w , w , h , h])
    load_image = load_img(image_path , target_size = (244 , 244))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr / 255.0
    xmin,xmax,ymin,ymax = labels[index]
    nxmin , nxmax = xmin/w , xmax/w
    nymin , nymax = ymin/h , ymax/h
    label_norm = (nxmin , nxmax ,nymin ,nymax)
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [5]:
#loading the model

import tensorflow as tf

model = tf.keras.models.load_model('./models/object_detection.h5')

In [15]:
test_data = np.array(data , dtype = np.float32)

In [16]:
predictions = model.predict(test_data)

2/2 [==============================] - 15s 1s/step


In [17]:
predictions[:5]

array([[0.3019115 , 0.653579  , 0.5448914 , 0.6567637 ],
       [0.25217223, 0.4930503 , 0.7284467 , 0.82579064],
       [0.3441298 , 0.63881433, 0.42463395, 0.53476083],
       [0.34772   , 0.7103614 , 0.42368323, 0.5629044 ],
       [0.4786798 , 0.8154831 , 0.50838894, 0.65856004]], dtype=float32)

In [18]:
#w w h h
original_dim[:5]

[[500, 500, 378, 378],
 [800, 800, 530, 530],
 [642, 642, 400, 400],
 [959, 959, 959, 959],
 [500, 500, 268, 268]]

In [19]:
denormalised_coord = predictions * original_dim
denormalised_coord[:5]

array([[150.95575154, 326.78949833, 205.96895564, 248.25666833],
       [201.73778534, 394.44024563, 386.07676268, 437.66904116],
       [220.93133211, 410.11879992, 169.85358   , 213.90433311],
       [333.46347728, 681.23660284, 406.31221384, 539.8253364 ],
       [239.339903  , 407.74154663, 136.24823499, 176.49409008]])

In [20]:
original_license_plate_coord = df.iloc[:,1:].values
original_license_plate_coord[:5]

array([[145, 319, 200, 244],
       [186, 395, 376, 430],
       [211, 406, 161, 202],
       [315, 665, 442, 531],
       [248, 421, 121, 174]], dtype=int64)

In [21]:
import math

In [22]:
def calculateIOU(pt1, pt2, pt3, pt4):
    X0A, Y0A = pt1
    X1A, Y1A = pt2
    X0B, Y0B = pt3
    X1B, Y1B = pt4

    X01 = max(X0A, X0B)
    X11 = min(X1A, X1B)
    Y01 = max(Y0A, Y0B)
    Y11 = min(Y1A, Y1B)

    intersection_area = max(0, X11 - X01) * max(0, Y11 - Y01)

    area_box_a = abs(X1A - X0A) * abs(Y1A - Y0A)
    area_box_b = abs(X1B - X0B) * abs(Y1B - Y0B)

    union_area = area_box_a + area_box_b - intersection_area

    IOU = intersection_area / union_area
    return IOU

In [23]:
IOU_results = []
for i in range(len(original_license_plate_coord)):
    predicted_box = denormalised_coord[i]
    original_box = original_license_plate_coord[i]
    
    X0A = original_box[0]
    X1A = original_box[1]
    Y0A = original_box[2]
    Y1A = original_box[3]
    
    X0B = predicted_box[0]
    X1B = predicted_box[1]
    Y0B = predicted_box[2]
    Y1B = predicted_box[3]
    
    pt1 = (X0A, Y0A)
    pt2 = (X1A, Y1A)
    pt3 = (X0B, Y0B)
    pt4 = (X1B, Y1B)
    
    res = calculateIOU(pt1, pt2, pt3, pt4)
    IOU_results.append(res)

In [24]:
print(IOU_results[:5])

[0.7345261737177767, 0.663132242422463, 0.5731743749445101, 0.6137582536622237, 0.608164593435467]


In [25]:
total_iou = 0
for i in IOU_results:
    total_iou += i
avg_iou = total_iou / len(IOU_results)

In [26]:
print('Result of the IOU is : ' , round(avg_iou , 2))
print('Percentage of overlapping : ' , round(avg_iou , 2) * 100)

Result of the IOU is :  0.68
Percentage of overlapping :  68.0


In [27]:
#Finding the images that model failed to predict
threshold = 0.5
modified_sum_iou = 0
failed_records = 0
failed_records_index = []
for i in range(len(IOU_results)):
    if IOU_results[i] > threshold:
        modified_sum_iou += IOU_results[i]
    else:
        failed_records += 1
        failed_records_index.append(i)
new_len = len(IOU_results) - failed_records
avg_IOU = modified_sum_iou / new_len
avg_IOU = round(avg_IOU , 2)
print('New IOU : ' , avg_IOU * 100)
print('No of failed records : ' , failed_records)

New IOU :  70.0
No of failed records :  3


In [28]:
#list of failed records
#print(failed_records_index)
for i in failed_records_index:
    print(df.iloc[i][0])

./images\N19.xml
./images\N24.xml
./images\N163.xml


In [29]:
def calculate_tp_fn(predicted_boxes, actual_boxes, threshold , IOU_results):
    true_positives = 0
    false_negatives = 0
    
    detected = [False] * len(actual_boxes)
    max_iou = 0
    max_iou_index = -1
    
    for i in range(len(actual_boxes)):
        if(IOU_results[i] > max_iou):
            max_iou += 1
            max_iou_index = i
        if(IOU_results[i] >= threshold):
            true_positives += 1
        else:
            false_negatives += 1
    return true_positives, false_negatives


In [30]:
actual_boxes = original_license_plate_coord
predicted_boxes = denormalised_coord
threshold = 0.5
true_positives , false_negatives = calculate_tp_fn(predicted_boxes , actual_boxes , threshold , IOU_results)
print("Percentage of true positive records : "  , round(true_positives / len(actual_boxes) * 100 , 2))
print("Percentage of false negative records : "  , round(false_negatives / len(actual_boxes) * 100 , 2))

Percentage of true positive records :  92.68
Percentage of false negative records :  7.32


In [31]:
print('Detection Accuracy of the model : ' , round(true_positives / len(actual_boxes) * 100 , 2))

Detection Accuracy of the model :  92.68


In [32]:
import pytesseract as pt

In [33]:
items_to_remove = ['./images\\N19.xml', './images\\N24.xml', './images\\N163.xml']
for item in items_to_remove:
    if item in path:
        path.remove(item)

In [34]:
items_to_remove = ['./images\\N19.jpeg', './images\\N24.jpeg', './images\\N163.jpeg']
for item in items_to_remove:
    if item in images_path:
        images_path.remove(item)

In [35]:
#using the pipeline created in make_predictions
#creatig a final pipeline

predicted_license_plate_numbers=[]

def object_detection(path):
    image = load_img(path) #PIL object
    image = np.array(image , dtype = np.uint8)
    #data preprocessing
    resized_image = load_img(path , target_size=(244 , 244))
    image_arr_244 = img_to_array(resized_image) / 255.0
    #array is normalised above
    h , w , d = image.shape
    test_arr = image_arr_244.reshape(1 , 244 , 244 , 3)
    #making predictions
    coords = model.predict(test_arr)
    denorm = np.array([w,w,h,h])
    #denormalising
    denormalised_coords = coords * denorm
    denormalised_coords = denormalised_coords.astype(np.int32)
    xmin , xmax , ymin , ymax = denormalised_coords[0]
    #pt1 = (xmin , ymin)
    #pt2 = (xmax , ymax)
    #cv2.rectangle(image , pt1 , pt2 , (0 , 255 , 0) , 3)
    return image , denormalised_coords


def OCR(path , cords):
    xmin , xmax , ymin , ymax = cords[0]
    img = np.array(load_img(path))
    roi = img[ymin - 5:ymax + 5, xmin - 5:xmax + 5]
    ImagePreprocessingBeforeOCR(roi)
    
def ImagePreprocessingBeforeOCR(roi_image):
    resize_test_license_plate = cv2.resize(roi_image, None, fx = 2, fy = 2,  interpolation = cv2.INTER_CUBIC)
    grayscale_resize_test_license_plate = cv2.cvtColor(resize_test_license_plate, cv2.COLOR_BGR2GRAY)
    gaussian_blur_license_plate = cv2.GaussianBlur(grayscale_resize_test_license_plate, (5, 5), 0) 
    new_predicted_result = pt.image_to_string(gaussian_blur_license_plate, lang='eng', config='--oem 3 -l eng --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    predicted_license_plate_numbers.append(new_predicted_result)
    
    
for i in images_path:
    image , cords = object_detection(i)
    OCR(i , cords)

1/1 [==============================] - 0s 364ms/step


In [36]:
predicted_license_plate_numbers[:5]

['KL25 B2001\n\x0c',
 'A03AB3380\n\x0c',
 'KL59K2473\n\x0c',
 'XL10AV 6633\n\x0c',
 'SKLOILAZ2\n\x0c']

In [37]:
cleaned_data = []
for string_data in predicted_license_plate_numbers:
    cleaned_string = string_data.replace('\n', '').replace('\x0c', '').replace(' ' , '')
    cleaned_data.append(cleaned_string)

In [39]:
cleaned_data[:5]

['KL25B2001', 'A03AB3380', 'KL59K2473', 'XL10AV6633', 'SKLOILAZ2']

In [40]:
predictions_df = pd.DataFrame(cleaned_data)
predictions_df.to_csv('predicted_license_plate_results.csv',index=False)

In [47]:
predictions_df = pd.read_csv('predicted_license_plate_results.csv' , names=['Predicted'])

In [48]:
actual_results_df = pd.read_csv('actual_license_plate_results.csv' , names=['Actual'])

In [50]:
def calculate_accuracy(predicted, actual):
    count = 0
    for pred_char in predicted:
        if pred_char in actual:
            count += 1
    return count / len(actual)

# Calculate accuracy for each row
accuracies = []
for pred_plate, actual_plate in zip(predictions_df['Predicted'], actual_results_df['Actual']):
    accuracy = calculate_accuracy(pred_plate, actual_plate)
    accuracies.append(accuracy)

# Calculate average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print("Average Accuracy:", average_accuracy * 100)

Average Accuracy: 85.08597883597885
